#LIBRARY

In [ ]:
%%capture
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


#LOAD DATA

In [ ]:
from google.colab import drive
!gdown "1rD-_mHaqds2plYf3R20xe42NKKAEku49"

Downloading...
From: https://drive.google.com/uc?id=1rD-_mHaqds2plYf3R20xe42NKKAEku49
To: /content/databarulagi.csv
100% 48.8M/48.8M [00:00<00:00, 115MB/s]


In [ ]:
# Load Data
df = pd.read_csv("databarulagi.csv")
df.head()

,Unnamed: 0,index,full_text,sentimen,keyword,Cleaning,Casefolding,Alayfix,Tokenized,Change Word,Removed Stopwords,Untokenized,Stemmed
0,2906,3088,6342E+18;Surya Paloh Tak Masalah Jika Anies Ba...,1,anies,e surya paloh tak masalah jika anies baswedan ...,e surya paloh tak masalah jika anies baswedan ...,e surya paloh masalah anies baswedan pilih ahy...,"['e', 'surya', 'paloh', 'masalah', 'anies', 'b...","['e', 'surya', 'paloh', 'masalah', 'Anies', 'B...","['paloh', 'Anies', 'Baswedan', 'pilih', 'agus'...",paloh Anies Baswedan pilih agus harimurti yudh...,paloh anies baswedan pilih agus harimurti yudh...
1,8722,9612,"1708146700450440000 """"@lhayesno @ganjarpranowo...",1,anies,boleh tdk sy piniam narasi,boleh tdk sy piniam narasi,tdk sy piniam narasi,"['tdk', 'sy', 'piniam', 'narasi']","['tidak', 'saya', 'piniam', 'narasi']","['piniam', 'narasi']",piniam narasi,piniam narasi
2,13311,14840,@BanyuKirik @Yuliadi_Mr @ganjarpranowo @KAI121...,1,ganjar,jadi buzzer anies aja drun,jadi buzzer anies aja drun,jadi buzzer anies aja drun,"['jadi', 'buzzer', 'anies', 'aja', 'drun']","['jadi', 'buzzer', 'Anies', 'saja', 'kadrun']","['buzzer', 'Anies', 'kadrun']",buzzer Anies kadrun,buzzer anies kadrun
3,28000,33979,"Bansos itu disalurkan kepada masyarakat, agar ...",1,bansos,bansos itu salur kepada masyarakat agar bisa g...,bansos itu salur kepada masyarakat agar bisa g...,bansos salur masyarakat guna untuk penuh butuh...,"['bansos', 'salur', 'masyarakat', 'guna', 'unt...","['bantuan', 'sosial', 'salur', 'masyarakat', '...","['bantuan', 'sosial', 'salur', 'masyarakat', '...",bantuan sosial salur masyarakat berguna penuh ...,bantu sosial salur masyarakat guna penuh butuh...
4,42135,48916,"1597603508618160000 """"Survei Terbaru Indopol",1,anies,survei baru indopol,survei baru indopol,survei baru indopol,"['survei', 'baru', 'indopol']","['survei', 'baru', 'indopol']",['survei'],survei,survei


In [ ]:
filtered_df = df[df['sentimen'].isin([0, 2])]
df = filtered_df
df['sentimen'].value_counts()

2    21866
0    15525
Name: sentimen, dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df_copy = df[['Stemmed','sentimen']]
df_copy.head()

,Stemmed,sentimen
18616,viral generasi bohong,0
18617,rusak nama anies baswedan jelek jelek nama anies,0
18618,bengkak,0
18619,kafir halang allah,0
18620,wajib presiden rakyat indonesia dukung preside...,0


In [ ]:
review = df["Stemmed"]
label = df['sentimen']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectors= TfidfVectorizer(min_df=7, ngram_range=(1,2), max_features = 5000, norm='l1')
vectors

TfidfVectorizer(max_features=5000, min_df=7, ngram_range=(1, 2), norm='l1')

In [ ]:
tfidf = vectors.fit_transform(review.astype(str))
tfidf

<37391x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 416095 stored elements in Compressed Sparse Row format>

In [ ]:
df_tfidf = tfidf.copy()

In [ ]:
label.value_counts()

2    21866
0    15525
Name: sentimen, dtype: int64

In [ ]:
x = tfidf.copy()
y = df['sentimen']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state=42)

In [ ]:
y_train.value_counts()

2    19688
0    13963
Name: sentimen, dtype: int64

In [ ]:
y_test.value_counts()

2    2178
0    1562
Name: sentimen, dtype: int64

In [ ]:
print(x_train.shape)

(33651, 5000)


## LOAD CORPUS WORD2VEC

In [ ]:
from gensim.models import Word2Vec
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
#berita+tweet
#!gdown "1UbgKJqGaBo0nUjB1KdGszVdUpZXtciOh"
#!gdown "1CUkGeGAxJIX_ObHzKSdS2epjYSsflmBJ"
#!gdown "1gFNEEIrwwlsoxi0FNKVJLkrijhzncPmA"
#berita
!gdown "1lOAO2PE1vm-CWg1dGfv8b4X12vWirLFU"
!gdown "1kyNC3SOjerhvGhFwsV3B3AdOUqOBBm62"
!gdown "11gE4o7nrT5x6FSw-cUG-XoY3FIEi52vi"
#tweet
#!gdown "1lR6aLe-FZg_riF45zGtBrP91qRWk17h4"

Downloading...
From: https://drive.google.com/uc?id=1lOAO2PE1vm-CWg1dGfv8b4X12vWirLFU
To: /content/w2v_berita.model
100% 8.23M/8.23M [00:00<00:00, 68.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1kyNC3SOjerhvGhFwsV3B3AdOUqOBBm62
From (redirected): https://drive.google.com/uc?id=1kyNC3SOjerhvGhFwsV3B3AdOUqOBBm62&confirm=t&uuid=a2cb41fa-4e4c-4c89-a5ce-08c85ae5d6cc
To: /content/w2v_berita.model.wv.vectors.npy
100% 180M/180M [00:01<00:00, 112MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=11gE4o7nrT5x6FSw-cUG-XoY3FIEi52vi
From (redirected): https://drive.google.com/uc?id=11gE4o7nrT5x6FSw-cUG-XoY3FIEi52vi&confirm=t&uuid=1ac417e2-13e7-4e26-a665-03efffaf1d65
To: /content/w2v_berita.model.syn1neg.npy
100% 180M/180M [00:02<00:00, 82.1MB/s]


In [ ]:
w2v_model = Word2Vec.load('w2v_berita.model')

# MODEL WORD2VEC

In [ ]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try:
            sim_word = w2v_model.wv.similar_by_word(col, topn = 1) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [ ]:
feature_ef = vectors.get_feature_names_out()

In [ ]:
# Membuat dataframe data test & train
df_x_train = pd.DataFrame(x_train.todense(), columns = feature_ef)
df_x_test = pd.DataFrame(x_test.todense(), columns = feature_ef)

In [ ]:
from tqdm import tqdm
dfexp_x_train= feature_expansion(df_x_train, feature_ef)

100%|██████████| 5000/5000 [01:19<00:00, 62.76it/s] 


In [ ]:
dfexp_x_test = feature_expansion(df_x_test, feature_ef)

100%|██████████| 5000/5000 [01:09<00:00, 71.94it/s]


In [ ]:
x_train_model = np.expand_dims(dfexp_x_train,1)
x_test_model =np.expand_dims(dfexp_x_test,1)

In [ ]:
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values
y_test

array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [ ]:
print(x_train_model.shape)

(33651, 1, 5000)


# GRU MODEL

In [ ]:
from keras.layers import GRU
from keras.models import Sequential
from keras.layers import Dense,Bidirectional,Embedding,Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
# model
# del model
def GRU_model(x_train_c, y_train_c, x_test_c, y_test_c):
    input_shape = x_train_c.shape
    print("Shape of x_train_c:", x_train_c.shape)


    model2 = Sequential()
    model2.add(GRU(256, return_sequences=True, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(GRU(128, return_sequences=True, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(GRU(64, return_sequences=False, activation='relu'))
    model2.add(Dropout(0.5))
    #model2.add(Dense(64,activation='tanh'))
    #model2.add(Dense(64,activation='tanh'))
    #model2.add(Dense(10,activation='tanh'))
    model2.add(Dense(2,activation='softmax'))
    model2.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
    model2.build(input_shape)

    batch_size = 128

    es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
    mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

    history = model2.fit(x_train_c, y_train_c, epochs = 10, batch_size=batch_size,
              validation_data=(x_test_c, y_test_c), verbose = 1,
              callbacks=[EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001)])

    y_pred = model2.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)

    acc = accuracy_score(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    prec = precision_score(y_test_cat, y_pred_cat, average="macro")

    print(accuracy_score(y_test_cat, y_pred_cat))
    print(classification_report(y_test_cat, y_pred_cat))
    print("Precision: ",precision_score(y_test_cat, y_pred_cat, average="macro"))
    print("Recall: ",recall_score(y_test_cat, y_pred_cat, average="macro"))
    print("F1: ",f1_score(y_test_cat, y_pred_cat,  average="macro"))
    print("Accuracy: ",accuracy_score(y_test_cat, y_pred_cat))

    return [acc, f1, recall, prec]


In [ ]:
GRU_model(x_train_model, y_train, x_test_model, y_test)

Shape of x_train_c: (33651, 1, 5000)
Epoch 1/10
263/263 [==============================] - 31s 100ms/step - loss: 0.6776 - accuracy: 0.5844 - val_loss: 0.6633 - val_accuracy: 0.5824
Epoch 2/10
263/263 [==============================] - 31s 117ms/step - loss: 0.5318 - accuracy: 0.7294 - val_loss: 0.4295 - val_accuracy: 0.8171
Epoch 3/10
263/263 [==============================] - 28s 108ms/step - loss: 0.4247 - accuracy: 0.8169 - val_loss: 0.4040 - val_accuracy: 0.8249
Epoch 4/10
263/263 [==============================] - 25s 97ms/step - loss: 0.3968 - accuracy: 0.8357 - val_loss: 0.3979 - val_accuracy: 0.8278
Epoch 5/10
263/263 [==============================] - 26s 100ms/step - loss: 0.3791 - accuracy: 0.8431 - val_loss: 0.3980 - val_accuracy: 0.8286
Epoch 6/10
263/263 [==============================] - 26s 100ms/step - loss: 0.3630 - accuracy: 0.8529 - val_loss: 0.4028 - val_accuracy: 0.8251
Epoch 7/10
263/263 [==============================] - 28s 108ms/step - loss: 0.3522 - accuracy

[0.8286096256684492, 0.823522245226593, 0.822970421241868, 0.8241198818514966]

In [ ]:
%%time
hasil = []

# looping dari 1 - 5 (5 kali run)
for i in range(1,6):
    hasil.append([str(i)] + GRU_model(x_train_model, y_train, x_test_model, y_test))

Shape of x_train_c: (33651, 1, 5000)
Epoch 1/10
263/263 [==============================] - 31s 97ms/step - loss: 0.6731 - accuracy: 0.5858 - val_loss: 0.6253 - val_accuracy: 0.6021
Epoch 2/10
263/263 [==============================] - 27s 103ms/step - loss: 0.5045 - accuracy: 0.7594 - val_loss: 0.4167 - val_accuracy: 0.8243
Epoch 3/10
263/263 [==============================] - 25s 96ms/step - loss: 0.4191 - accuracy: 0.8189 - val_loss: 0.4015 - val_accuracy: 0.8283
Epoch 4/10
263/263 [==============================] - 26s 101ms/step - loss: 0.3956 - accuracy: 0.8352 - val_loss: 0.3985 - val_accuracy: 0.8299
Epoch 5/10
263/263 [==============================] - 25s 97ms/step - loss: 0.3769 - accuracy: 0.8462 - val_loss: 0.3988 - val_accuracy: 0.8275
Epoch 6/10
263/263 [==============================] - 26s 99ms/step - loss: 0.3609 - accuracy: 0.8557 - val_loss: 0.4021 - val_accuracy: 0.8214
Epoch 7/10
263/263 [==============================] - 26s 97ms/step - loss: 0.3471 - accuracy: 0.

In [ ]:
df = pd.DataFrame(hasil, columns =['itr', 'acc', 'f1', 'recall', 'precision'])
df

,itr,acc,f1,recall,precision
0,1,0.829412,0.824905,0.825379,0.824464
1,2,0.831283,0.827997,0.831060,0.826153
2,3,0.830749,0.827555,0.830873,0.825659
3,4,0.827273,0.822456,0.822456,0.822456
4,5,0.834759,0.831425,0.834226,0.829639


In [ ]:
print("Accuracy : %.2f" % (df['acc'].mean()*100))
print("F1-Score : %.2f" % (df['f1'].mean()*100))

Accuracy : 83.07
F1-Score : 82.69
